In [1]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing import image

In [8]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"
images_data_size = 10000 

width = 178
height = 218
x_ = 100                            # new value of x
image_size_ratio = width / height    # dimensions ratio
y_ = int(image_size_ratio * x_)  
df_org = pd.read_csv(attr_data_path)
df = df_org[:images_data_size]
df = df.replace(-1,0)
#print(df.head(5))
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)
def image_from_index(index, size=(width,height), path=images_data_path):
    path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)

    return image_array
images = images_data # list of array of images
labels = df[["image_id", "Bangs", "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(4))
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)
X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(1))
print("Samples in Validation Data:",len(X_val))
print("Samples in Test Data:",len(X_test))

Samples Training Set: 7000
Samples in Validation Data: 1500
Samples in Test Data: 1500


In [25]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(tf.keras.Input(shape=(height, width,1)))
model.add(tf.keras.layers.Normalization())

#conv2d
model.add(Conv2D(filters=16,kernel_size=(8,8),padding = 'valid'))
#conv2d-1
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid'))
model.add(tf.keras.layers.Normalization())

#conv2d-2
model.add(Conv2D(filters=12, kernel_size=(4, 4), strides=1, padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=16,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=16,kernel_size=(2,2),padding = 'same'))
model.add(tf.keras.layers.Normalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=80, kernel_size=(3,3), padding='same'))
model.add(Conv2D(filters=32, kernel_size=(2,2), padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=3))

model.add(tf.keras.layers.Normalization())

model.add(Flatten())
model.add(Dense(64))
model.add(Dense(48))
model.add(Dense(16))
model.add(Dense(4, activation='sigmoid'))
original_image_size = (width,height)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 218, 178, 1)      3         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 211, 171, 16)      1040      
                                                                 
 conv2d_1 (Conv2D)           (None, 207, 167, 16)      6416      
                                                                 
 normalization_1 (Normalizat  (None, 207, 167, 16)     33        
 ion)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 207, 167, 12)      3084      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 103, 83, 12)      0

In [26]:
BATCH_SIZE = (6)
model.compile(optimizer = 'adam',loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
training_process = model.fit(X_train,y_train,epochs = (4),validation_data = (X_val,y_val),batch_size=BATCH_SIZE,shuffle = True)
model.evaluate(X_val,y_val)

Epoch 1/4
1167/1167 [==============================] - 42s 35ms/step - loss: 2.1149 - precision: 0.4292 - recall: 0.3363 - val_loss: 0.4946 - val_precision: 0.4044 - val_recall: 0.7067
Epoch 2/4
1167/1167 [==============================] - 41s 35ms/step - loss: 0.3519 - precision: 0.6247 - recall: 0.4151 - val_loss: 0.3799 - val_precision: 0.5396 - val_recall: 0.5074
Epoch 3/4
1167/1167 [==============================] - 42s 36ms/step - loss: 0.3256 - precision: 0.6649 - recall: 0.4318 - val_loss: 0.3419 - val_precision: 0.5904 - val_recall: 0.5168
Epoch 4/4
47/47 [==============================] - 2s 39ms/step - loss: 0.3694 - precision: 0.5497 - recall: 0.5875


[0.3693610727787018, 0.549651563167572, 0.5875232815742493]

In [22]:
model.evaluate(X_val,y_val)

47/47 [==============================] - 2s 39ms/step - loss: 0.4426 - precision: 0.4646 - recall: 0.6350


[0.44264888763427734, 0.46457764506340027, 0.6350092887878418]

In [9]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"

tf.keras.backend.clear_session()
width = 178
height = 218
df_org = pd.read_csv(attr_data_path)

df = df_org[190000:202600]
df = df.replace(-1,0)
#print(df.head(5))
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)
def image_from_index(index, size=(width,height), path=images_data_path):
    path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)

    return image_array
images = images_data # list of array of images
labels = df[["image_id", "Bangs", "Straight_Hair", "Wavy_Hair", "Wearing_Hat"]]  # dataframe of image features
labels=labels.drop(["image_id"],axis=1)



In [10]:
for i in range(1, 19):
    tf.keras.backend.clear_session()
    model1 = tf.keras.models.load_model('cnn8_checkpoints/iter'+str(i))
    model1.evaluate(images,labels)

394/394 [==============================] - 13s 32ms/step - loss: 0.3009 - precision: 0.7161 - recall: 0.5216 - binary_accuracy: 0.8682
